BIO data as per CoNNL should be in text format. Here we convert our csv file to `txt` format.

In [14]:
import pandas as pd

In [33]:
import string

def convert_to_bio_format(df):
    data = []
    for i, row in df.iterrows():
        query, email, mobile, first_name, last_name = row
        
        entities = {
            'Email': email,
            'Mobile': mobile,
            'FirstName': first_name,
            'LastName': last_name,
        }
        
        tokens = query.split()
        
        bio_tags = []
        for token in tokens:
            clean_token = token.strip(string.punctuation)
            for entity, value in entities.items():
                if clean_token in value:
                    if value.startswith(clean_token):
                        bio_tags.append('B-' + entity)
                        break
                    else:
                        bio_tags.append('I-' + entity)
                        break
            else:
                bio_tags.append('O')
                
        data.append(list(zip([token.strip(string.punctuation) for token in tokens], bio_tags)))
        
    return data

In [34]:
df = pd.read_csv('./data/shuffled_data.csv')
bio_data = convert_to_bio_format(df)

In [39]:
with open('./data/ner_data.txt', 'w') as f:
    for sentence in bio_data:
        for word, tag in sentence:
            f.write(f'{word} {tag}\n')
        f.write('\n')  # Write a blank line to separate sentences


In [37]:
# Flatten the data and create a DataFrame
flat_data = [(i, word, tag) for i, sentence in enumerate(bio_data, 1) for word, tag in sentence]
df = pd.DataFrame(flat_data, columns=['SentenceID', 'Word', 'Tag'])

# Save the DataFrame to a CSV file
df.to_csv('ner_data.csv', index=False)